<a href="https://colab.research.google.com/github/Qudsia-jabeen20/Facial-Expression-Recognition/blob/main/emotion_detection_with_facial_expression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import sys
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing import image

import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("ananthu017/emotion-detection-fer")

print("Path to dataset files:", path)

In [ ]:


print("Files in dataset directory:")
print(os.listdir(path))


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

img_size = (48, 48)
batch_size = 32

train_dir = os.path.join(path, 'train')
test_dir = os.path.join(path, 'test')

train_gen = ImageDataGenerator(rescale=1./255).flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    color_mode='grayscale'
)

test_gen = ImageDataGenerator(rescale=1./255).flow_from_directory(
    test_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    color_mode='grayscale'
)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(48, 48, 1)),
    MaxPooling2D(2, 2),

    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2, 2),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(train_gen.num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


In [ ]:
model.fit(train_gen, validation_data=test_gen, epochs=10)


In [ ]:
loss, acc = model.evaluate(test_gen)
print(f"Test Accuracy: {acc:.2f}")


In [ ]:
loss, accuracy = model.evaluate(test_gen)
print(f"Test Accuracy: {accuracy * 100:.2f}%")


In [ ]:
from google.colab import files
uploaded = files.upload()

for filename in uploaded.keys():
    img = image.load_img(filename, color_mode='grayscale', target_size=(48, 48))
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    pred = model.predict(img_array)
    predicted_label = np.argmax(pred)
    print("Predicted Emotion:", class_labels[predicted_label])


In [ ]:
keras.saving.save_model(model, 'my_model.keras')


In [ ]:
!pip install gradio --quiet

In [ ]:
import gradio as gr
from tensorflow.keras.models import load_model
import cv2


model = load_model("emotion_model.h5")
emotion_labels = ['angry', 'disgust', 'fearful', 'happy', 'neutral', 'sad', 'surprised']


def predict_emotion(pil_img):
    try:
        img = np.array(pil_img)
        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        face = cv2.resize(gray, (48, 48))
        face = face.astype("float32") / 255.0
        face = np.expand_dims(face, axis=-1)
        face = np.expand_dims(face, axis=0)

        prediction = model.predict(face)[0]
        top_emotions = {emotion_labels[i]: float(prediction[i]) for i in range(len(prediction))}
        return top_emotions

    except Exception as e:
        return f"Error: {e}"
        gr.Interface(
    fn=predict_emotion,
    inputs=gr.Image(source="webcam", tool=None),
    outputs=gr.Label(num_top_classes=3),
    live=True,
    title="Real-Time Emotion Detection"
).launch()
